<a href="https://colab.research.google.com/github/will-gun/stock-data/blob/master/collectTaiwanStockData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
from io import StringIO
import requests
import pandas as pd
import numpy as np
import pickle

def create_today_timestamp():
    today = time.strftime("%Y-%m-%d",time.gmtime())
    return int(time.mktime(time.strptime(today, "%Y-%m-%d")))

def create_timestamp_from_today(n):
    today = create_today_timestamp()
    return today + n*24*3600
tomorrow_timestamp = create_timestamp_from_today(1)
print(tomorrow_timestamp)

def create_tw_stock_info_list():
    res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
    df = pd.read_html(res.text)[0]
    print(df)
    df.columns = df.iloc[0]
    df = df.iloc[2:900]
    df = df.dropna(thresh=3, axis=0).dropna(thresh=3, axis=1)
    #df = df.dropna(how='any')
    df = df.reset_index(drop=True)
    print(df)
    new_df = df['有價證券代號及名稱'].str.replace(u'\u3000',' ').str.split(u' ',expand=True)
    print(new_df)
    new_df.columns = ['Ticker', 'StockName']
    new_df['Sector'] = df['產業別']
    return new_df

tw_stock_info_df = create_tw_stock_info_list()
tw_stock_info_df

#min = np.array([[100000.0,0],[100000.0,0],[100000.0,0],[100000.0,0],[100000.0,0],[100000.0,0]])
min = np.empty((0,3))
min_temp = np.array( [[10000000.0, 10.0, 10.0]] )
day = 750
day_1 = day-1

stock_df = pd.DataFrame()
ticker_list = tw_stock_info_df['Ticker']
#ticker_list = tw_stock_info_df['Ticker'].tail(10)
for ticker in ticker_list:
    print('## Info: Download Ticker '+ticker+'!')
    site = "https://query1.finance.yahoo.com/v7/finance/download/"+ticker+".TW?period1=0&period2="+str(tomorrow_timestamp)+"&interval=1d&events=history&crumb=hP2rOschxO0"
    try:
        response = requests.get(site)
        tmp_df = pd.read_csv(StringIO(response.text))
        tmp_df['Ticker'] = ticker
        #stock_df = pd.concat([stock_df,tmp_df],axis=0)
                  
    except:
        print('## Warning: Ticker '+ticker+' is failed!')

    if len(tmp_df.index) > day:
        tmp_df = tmp_df[len(tmp_df.index)-day:]
        tmp_df = tmp_df.reset_index(drop=True)
        #print(tmp_df)
        abc = (tmp_df.at[day_1,'Adj Close'] - tmp_df.at[0,'Adj Close']) / day_1
        if (tmp_df.at[day_1,'Adj Close'] - tmp_df.at[0,'Adj Close'])/tmp_df.at[0,'Adj Close'] > 0.3:
          mmin = 0
          for i in range(day):
            mmin += abs(tmp_df.at[i,'Adj Close']-(abc*i+tmp_df.at[0,'Adj Close']))/(abc*i+tmp_df.at[0,'Adj Close'])
          min_temp[0,0]=mmin
          min_temp[0,1]=(tmp_df.at[day_1,'Adj Close'] - tmp_df.at[0,'Adj Close'])/tmp_df.at[0,'Adj Close']
          min_temp[0,2]=ticker
          min = np.append(min, min_temp, axis=0)
          #min=min[min[:,0].argsort()]
        
print(min)
max=min[(-min[:,1]).argsort()]
min=min[min[:,0].argsort()]

x,y,z = max.T
X,Y,Z = min.T

z1 = np.array([0])
Z1 = np.array([1])
res = np.array([0])

for i in range(len(z)):
  z1 = np.append(z1, z[i])
  a = np.intersect1d(z1, Z1)
  if len(a) > 0:
    res = np.append(res, a)
    z1 = np.delete(z1, [len(z1)-1])
  Z1 = np.append(Z1, Z[i])
  a = np.intersect1d(z1, Z1)
  if len(a) > 0:
    res = np.append(res, a)
    Z1 = np.delete(Z1, [len(Z1)-1])

print(res)

#stock_df = stock_df.reset_index(drop=True)
#stock_df = stock_df[['Date','Ticker','Open','High','Low','Close','Adj Close','Volume']]
#print(stock_df)
